## Data Extraction 

### This file is used as a playground in order to:

* Extract the data from postgres DB
* Inspect
* Create functions to create subsets of data to serve the dashboard

In [14]:
import pandas as pd
import psycopg2
import psycopg2.extras
import warnings
import os 
from dotenv import load_dotenv
warnings.filterwarnings('ignore')

load_dotenv(override=True, verbose=True)

DB_USER=os.getenv("DB_USER")
DB_PASSWORD=os.getenv("DB_PASSWORD")
DB_HOST=os.getenv("DB_HOST")
DB_NAME=os.getenv("DB_NAME")

def get_db_connection() -> psycopg2.extensions.connection:
    """ Create a connection for database postgres Aurora"""
    try:
        conn = psycopg2.connect(f"""
    dbname={DB_NAME}
    user={DB_USER} 
    password={DB_PASSWORD}
    host={DB_HOST}""")
        return conn 
    except:
        print("Error connecting to database.")

def query_executer(
    conn: psycopg2.extensions.connection, query: str, params: tuple = ()
) -> list:
    """An executor function for executing sql statements"""
    if conn != None:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(query, params)
            conn.commit()
            try:
                returned_data = cur.fetchall()
                return returned_data
            except:
                pass
    else:
        return "No connection"
        
conn = get_db_connection()

## Ingestions and inspections

### Look at the artwork Table

In [15]:
sql_1 = "SELECT * FROM artwork"
df_artwork = pd.read_sql(sql_1, conn)
print(df_artwork.info())
print(df_artwork.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6325 entries, 0 to 6324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   artwork_id      6325 non-null   int64 
 1   title           6325 non-null   object
 2   year_completed  6325 non-null   int64 
 3   department      6325 non-null   object
 4   artist_id       6325 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 247.2+ KB
None
      artwork_id                                              title  \
3758        3745  Untitled (plate, folio 49-50) from Poetry of S...   
2151        2145  Untitled, no. 4 of 34, from the illustrated bo...   
6124        6124            Intake Facility for an Anonymous Client   
904          903   Untitled from Fabricated Cultural Belief Systems   
3020        3011                              Naming Tokyo (part I)   
532          531                                  Kleiner Aufmarsch   
3051        3042  Low Tide Wande

### Inspection into artist table

In [16]:
sql_2 = "SELECT * FROM artist"
df_artist = pd.read_sql(sql_2, conn)
print(df_artist.info())
print(df_artist.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_id    155 non-null    int64 
 1   artist_name  155 non-null    object
 2   nationality  155 non-null    object
 3   gender       154 non-null    object
 4   year_start   155 non-null    int64 
 5   year_end     155 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 7.4+ KB
None
     artist_id        artist_name  nationality  gender  year_start  year_end
67        8383       Francis Alÿs      Belgian    Male        1959         0
97       28742    Paulina Olowska       Polish  Female        1976         0
63        8276  Thomas Hirschhorn        Swiss    Male        1957         0
96       28728      Paul Morrison      British    Male        1966         0
135      38858       Deana Lawson     American  Female        1979         0
111      30229           Mark Fox     American    M

In [17]:
# Join the tables
sql = """SELECT * FROM artwork 
            JOIN artist 
                ON artwork.artist_id = artist.artist_id"""
df_joined = pd.read_sql(sql, conn)

In [18]:
df_joined.shape

(6325, 11)

In [19]:
# drop duplicates
df_joined = df_joined[df_joined.duplicated() == False ]
df_joined.shape

(6325, 11)

### An inspection into the joined tables

In [20]:
print(df_joined.info())
(df_joined.sample(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6325 entries, 0 to 6324
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   artwork_id      6325 non-null   int64 
 1   title           6325 non-null   object
 2   year_completed  6325 non-null   int64 
 3   department      6325 non-null   object
 4   artist_id       6325 non-null   int64 
 5   artist_id       6325 non-null   int64 
 6   artist_name     6325 non-null   object
 7   nationality     6325 non-null   object
 8   gender          6287 non-null   object
 9   year_start      6325 non-null   int64 
 10  year_end        6325 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 593.0+ KB
None


,artwork_id,title,year_completed,department,artist_id,artist_id,artist_name,nationality,gender,year_start,year_end
3787,3773,Untitled from Andy Warhol Doesn't Play Second ...,2010,Drawings & Prints,28847,28847,Kelley Walker,American,Male,1969,0
4796,4785,Four Softground Etchings w/ Drypoint,2014,Drawings & Prints,6448,6448,Christopher Wool,American,Male,1955,0
3459,3448,Untitled from Secret Knots,2008,Drawings & Prints,6408,6408,Terry Winters,American,Male,1949,0
1002,1001,Untitled from Cubes in Color on Color,2003,Drawings & Prints,3528,3528,Sol LeWitt,American,Male,1928,2007
5251,5238,Independence/Nakba,2013,Photography,49682,49682,Fazal Sheikh,American,Male,1965,0
4867,4855,"Wir sind keine Enten auf dem Teich, wir sind S...",2015,Drawings & Prints,6288,6288,Lawrence Weiner,American,Male,1942,2021
3069,3060,Low Tide Wandering No. 48 (Wattwanderung No. 4...,2001,Drawings & Prints,5281,5281,Thomas Schütte,German,Male,1954,0
5254,5241,Independence/Nakba,2013,Photography,49682,49682,Fazal Sheikh,American,Male,1965,0
995,994,Untitled from Cubes in Color on Color,2003,Drawings & Prints,3528,3528,Sol LeWitt,American,Male,1928,2007
2066,2061,Untitled from Blueberry Surprise,2006,Drawings & Prints,7929,7929,Joseph Grigely,American,Male,1956,0


In [21]:
# Remove 'untitled' artwork
def drop_untitled_artwork(data: pd.DataFrame) -> pd.DataFrame:
    """Function to drop the titles with 'Untitled' in the title"""
    return data[~(data['title'].str.contains('Untitled'))]

df_processed = drop_untitled_artwork(df_joined)
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4124 entries, 0 to 6324
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   artwork_id      4124 non-null   int64 
 1   title           4124 non-null   object
 2   year_completed  4124 non-null   int64 
 3   department      4124 non-null   object
 4   artist_id       4124 non-null   int64 
 5   artist_id       4124 non-null   int64 
 6   artist_name     4124 non-null   object
 7   nationality     4124 non-null   object
 8   gender          4086 non-null   object
 9   year_start      4124 non-null   int64 
 10  year_end        4124 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 386.6+ KB


In [22]:
# Get unique departments

departments = list(df_processed['department'].unique())
print(departments)

['Drawings & Prints', 'Photography', 'Painting & Sculpture', 'Media and Performance', 'Architecture & Design']


In [23]:
# Filter by departments

def filter_by_department(data: pd.DataFrame ,department: str) -> pd.DataFrame:
    """Given a department, filter the dataframe to return results only from that department"""
    filtered_data = data[data['department'] == department]
    return filtered_data

# Inputs
department_test = departments[0]

print(f"\nFiltering by department: {department_test}\n")
print(filter_by_department(df_processed, department_test).head(10))


Filtering by department: Drawings & Prints

    artwork_id                                              title  \
0            1                                               Cane   
4            5                 Self-Portrait (for Parkett no. 60)   
7            8                     The Man in Black/Drone Harness   
8            9                                      Self-Portrait   
9           10                     Self-Portrait/Scribble/Etching   
10          11             Self-Portrait/Scribble/Etching Box Set   
11          12                         Self-Portrait/Pulp/Pochoir   
12          13             Ssblak!Ssblak!!Ssblakallblak! Wonder#9   
14          15   Elephant Women from the series Personal Articles   
15          16  "Hey If You're So Damn Smart, Why Can't You Ev...   

    year_completed         department  artist_id  artist_id      artist_name  \
0             2000  Drawings & Prints       4758       4758   Martin Puryear   
4             2000  Drawings & Prin

In [24]:
# Filter by data range for completed work

def filter_completed_work_by_date_range(data: pd.DataFrame, start_year: int, end_year: int) -> pd.DataFrame:
    """Given a start and end year, filter the data for completed work using these ranges"""
    filtered_data = data[(data['year_completed'] >= start_year) & (data['year_completed'] <= end_year)]
    return filtered_data

# Inputs
start = 2006
end = 2008

print(f"Testing for years between {start} - {end}\n")
print(filter_completed_work_by_date_range(df_processed, start, end).sample(15))

Testing for years between 2006 - 2008

      artwork_id                                              title  \
2706        2697                                         Opening Up   
1483        1478              De Theatercompagnie Driekoningenavond   
4234        4225  Woman Nr. 5 (Frau Nr. 5) from Women – Series C...   
5732        5733                 Donated Object: Over-the-Rhine Mug   
3984        3972                                 Turning Some Pages   
2010        2005                       Swatter (for Parkett no. 81)   
2052        2047  Security Barrier Type L from Security Barriers...   
3475        3464                                     Satin Operator   
3007        2998  Model K 1981 (Modell K 1981) from Architecture...   
2284        2278                                   Hours of the Day   
2105        2099                      "Dow Chemical Represents You"   
1672        1667                                   Knotted Graphs/3   
2123        2117                      

In [25]:
# Get number of artworks available in the collection

def get_the_number_of_artworks(data: pd.DataFrame) -> int:
    """Function to make return number of artworks available"""
    return len(data)

# Get test input for data
df_test = filter_by_department(df_processed, departments[0])
df_test = filter_completed_work_by_date_range(df_test, 2007, 2010)
num_of_artworks = get_the_number_of_artworks(df_test)

print(f"Number of unique artworks available in collection {num_of_artworks}\n")
print(df_test.head(10))

Number of unique artworks available in collection 570

      artwork_id                                             title  \
1557        1552                                            Within   
1565        1560                               James Brown is Dead   
1585        1580                      A Bookcase for onestar press   
1593        1588                                        I Love You   
1595        1590                                           Sampler   
1599        1594                            That Person's Workbook   
1600        1595               Spinner Winner (for Parkett no. 80)   
1601        1596                                   Reconstructions   
1603        1598                       Guardian of the Veil: Sekhu   
1604        1599  Guardian of the Veil: Adoration of Norman Mailer   

      year_completed         department  artist_id  artist_id  \
1557            2007  Drawings & Prints       2923       2923   
1565            2007  Drawings & Prints     

In [26]:
# See a demographic breakdown of artist information: Gender

def get_artists_gender_demographics(data: pd.DataFrame) -> pd.DataFrame:
    """Get the breakdown of artists gender demographics for given data"""
    # drop duplicate values for artists
    processed = data.drop_duplicates(subset=['artist_id'])
    gender_demographics = processed.groupby('gender').count().reset_index()
    gender_demographics['count'] = gender_demographics['artwork_id']
    
    return gender_demographics[['gender', 'count']]


print("Gender demographics for artists: \n")
print(get_artists_gender_demographics(df_test))

# To measure previous values against -> do numbers make sense?
total_artists = len(df_test['artist_name'].unique())
print(f"\nNumber of unique artists: {total_artists}")

Gender demographics for artists: 

   gender  count
0  Female     18
1    Male     46

Number of unique artists: 64


In [27]:
# See a demographic breakdown of artist information: Nationality

def get_artists_nationality_demographics(data: pd.DataFrame) -> pd.DataFrame:
    """Returns a data frame with artists nationality distribution"""
    # drop duplicate artists values
    processed = data.drop_duplicates(subset=['artist_id'])
    nationality_demographics = processed.groupby('nationality').count().reset_index()
    nationality_demographics['count'] = nationality_demographics['artwork_id']

    return nationality_demographics[['nationality', 'count']]


print("\nThe nationalities for the test data: \n")
print(get_artists_nationality_demographics(df_test))

# Test the count value:
sum_of_counts = get_artists_nationality_demographics(df_test)['count'].sum()
print(f"\nThe sum of all the count values: {sum_of_counts}")


The nationalities for the test data: 

      nationality  count
0        Albanian      1
1        American     43
2         British      2
3     Cameroonian      1
4        Canadian      1
5         Chinese      3
6       Colombian      2
7        Egyptian      1
8         Italian      1
9          Korean      1
10        Mexican      2
11      Pakistani      2
12         Polish      1
13  South African      1
14        Spanish      1
15          Swiss      1

The sum of all the count values: 64


In [28]:
print(df_joined['year_completed'].min())
print(df_joined['year_completed'].max())

2000
2021
